# **RAIN & Meta AI Academy: Functions Notebook**

This notebook serves as a proof of concept for the core components of our AI-powered notification system. We will test and validate each function individually before combining them into a final Streamlit application.

**Our Goals:**
1.  ✅ **AI Vision:** Use Groq Cloud and Meta's Llama 4 model to analyze an image for a face mask.
2.  ✅ **Email Alerts:** Send a simple text email using Python's built-in `smtplib`.
3.  ✅ **Email with Evidence:** Send an email with the analyzed image attached.
4.  ✅ **WhatsApp Alerts:** Send a WhatsApp message using the Twilio API.

### **Step 0: Setup and Installation**

First, let's install the necessary libraries and set up our environment variables.

#### **1. Install Libraries**

In [ ]:
!pip install groq python-dotenv twilio -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.0/136.0 kB 10.1 MB/s eta 0:00:00


In [ ]:
import os
import base64
import json
from dotenv import load_dotenv

load_dotenv()

False

In [ ]:
# .env file

# Groq Cloud API Key
GROQ_API_KEY="XXXXXXXXXXXXXXX"

# Email Credentials (for Gmail, use an "App Password")
SENDER_EMAIL="XXXXXXXXx@gmail.com"
EMAIL_PASSWORD="XXX XXX XXXX XXXXX"
RECEIVER_EMAIL="XXXXXXXXXX@gmail.com"


# Twilio Credentials
TWILIO_ACCOUNT_SID='XXXXXXXXXXXXXx'
TWILIO_AUTH_TOKEN='XXXXXXXXXXXXXXX'

TWILIO_WHATSAPP_NUMBER="+XXXXXXXXXX" # This is the Twilio Sandbox Number
MY_WHATSAPP_NUMBER="+XXXXXXXXXXX" # Your personal number, verified with the Sandbox

In [ ]:
import os
import base64
import json
from dotenv import load_dotenv

load_dotenv()

# Load Groq Credentials
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

# Load Email Credentials
SENDER_EMAIL = os.getenv("SENDER_EMAIL")
EMAIL_PASSWORD = os.getenv("EMAIL_PASSWORD")
RECEIVER_EMAIL = os.getenv("RECEIVER_EMAIL")

# Load Twilio Credentials
TWILIO_ACCOUNT_SID = os.getenv("TWILIO_ACCOUNT_SID")
TWILIO_AUTH_TOKEN = os.getenv("TWILIO_AUTH_TOKEN")
TWILIO_WHATSAPP_NUMBER = os.getenv("TWILIO_WHATSAPP_NUMBER")
MY_WHATSAPP_NUMBER = os.getenv("MY_WHATSAPP_NUMBER")

In [ ]:
from groq import Groq

# Function to encode the image to base64
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

def analyze_image_for_mask(image_path):
    """
    Analyzes an image using Groq's Llama 4 model to detect a face mask.

    Args:
        image_path (str): The path to the local image file.

    Returns:
        dict: A dictionary with the analysis results (e.g., {"mask_detected": false, "reason": "..."}).
    """
    try:
        client = Groq(api_key=GROQ_API_KEY)
        base64_image = encode_image(image_path)

        prompt = """
        Analyze the person in the provided image. Respond with a JSON object that has two keys:
        1. "mask_detected": a boolean (true if a face mask is worn correctly, otherwise false).
        2. "reason": a brief string explaining your choice in 1-2 sentences.

        Provide only the raw JSON object in your response.
        """

        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}",
                            },
                        },
                    ],
                }
            ],
            model="meta-llama/llama-4-scout-17b-16e-instruct",
            response_format={"type": "json_object"},
        )

        response_content = chat_completion.choices[0].message.content
        return json.loads(response_content)

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [ ]:
# --- DEMONSTRATION ---
print("--- Analyzing image WITH mask ---")
analysis_with_mask = analyze_image_for_mask("/content/images/image with mask.jpeg")
print(analysis_with_mask)
print("\n" + "="*30 + "\n")

print("--- Analyzing image WITHOUT mask ---")
analysis_without_mask = analyze_image_for_mask("/content/images/mother of dragons.jpeg")
print(analysis_without_mask)

--- Analyzing image WITH mask ---
{'mask_detected': True, 'reason': 'The person is wearing a face mask covering both nose and mouth.'}


--- Analyzing image WITHOUT mask ---
{'mask_detected': False, 'reason': 'The person in the image is not wearing a face mask.'}


### **Part 2: Sending Email with `smtplib`**

Now let's prove we can send email alerts.

#### **Function 1: Send a Basic Text Email**


In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def send_basic_email(subject, body):
    """Sends a simple text-based email."""
    try:
        msg = MIMEMultipart()
        msg['From'] = SENDER_EMAIL
        msg['To'] = RECEIVER_EMAIL
        msg['Subject'] = subject
        msg.attach(MIMEText(body, 'plain'))

        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(SENDER_EMAIL, EMAIL_PASSWORD)
        server.send_message(msg)
        server.quit()
        print("Successfully sent basic email!")
    except Exception as e:
        print(f"Failed to send email: {e}")

In [ ]:
# --- DEMONSTRATION ---
subject = "Test Alert: System is Online"
body = "This is a test email to confirm that the notification system is working correctly."
send_basic_email(subject, body)

Successfully sent basic email!


#### **Function 2: Send an Email with an Image Attachment**

This is crucial for providing evidence in our alerts.


In [ ]:
from email.mime.image import MIMEImage

def send_email_with_image(subject, body, image_path):
    """Sends an email with an image attachment."""
    try:
        msg = MIMEMultipart()
        msg['From'] = SENDER_EMAIL
        msg['To'] = RECEIVER_EMAIL
        msg['Subject'] = subject
        msg.attach(MIMEText(body, 'plain'))

        # Attach the image
        with open(image_path, 'rb') as f:
            img_data = f.read()
        image = MIMEImage(img_data, name=os.path.basename(image_path))
        msg.attach(image)

        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(SENDER_EMAIL, EMAIL_PASSWORD)
        server.send_message(msg)
        server.quit()
        print(f"Successfully sent email with image {image_path}!")
    except Exception as e:
        print(f"Failed to send email with image: {e}")

In [ ]:
# --- DEMONSTRATION ---
# We'll use the result from our earlier AI analysis
if analysis_without_mask and not analysis_without_mask.get("mask_detected"):
    subject = "ALERT: No Mask Detected!"
    body = f"An individual was detected without a mask.\n\nAI Analysis: {analysis_without_mask.get('reason')}"
    image_to_send = "/content/images/mother of dragons.jpeg"
    send_email_with_image(subject, body, image_to_send)

Successfully sent email with image /content/images/mother of dragons.jpeg!


### **Part 3: Sending WhatsApp Messages with Twilio**

Finally, let's test the WhatsApp functionality.

**Important:** Before running this, you must activate the Twilio Sandbox. Send a WhatsApp message with the code `join <your-sandbox-keyword>` to the number `+14155238886`. You can find your keyword in the Twilio console.

In [ ]:
!pip install twilio -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 55.9 MB/s eta 0:00:00


In [ ]:
from twilio.rest import Client

def send_whatsapp_message(body):
    """Sends a WhatsApp message using Twilio."""
    try:
        client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)
        message = client.messages.create(
            from_=f'whatsapp:{TWILIO_WHATSAPP_NUMBER}',
            body=body,
            to=f'whatsapp:{MY_WHATSAPP_NUMBER}'
        )
        print(f"WhatsApp message sent successfully! SID: {message.sid}")
    except Exception as e:
        print(f"Failed to send WhatsApp message: {e}")

In [ ]:
# --- DEMONSTRATION ---
if analysis_without_mask and not analysis_without_mask.get("mask_detected"):
    alert_body = f"🚨 *ALERT: No Mask Detected!* 🚨\n\n*Reason:* {analysis_without_mask.get('reason')}"
    send_whatsapp_message(alert_body)

WhatsApp message sent successfully! SID: SM010f0cb6a50a2fc8b122c034104bd38f


Author: Olufemi Victor